In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import scipy

from keras.models import Model, load_model, save_model
from keras.layers import Input, Dense, Embedding, concatenate, Add, Dropout, SpatialDropout1D, Conv1D, BatchNormalization, Activation
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, CuDNNLSTM
from keras.layers import AveragePooling1D, MaxPooling1D, Reshape, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Permute, Multiply
from keras import initializers, regularizers, constraints, optimizers, layers  
from keras.initializers import *
from keras.initializers import Constant
from keras.preprocessing import text, sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot  

from keras.utils import to_categorical 
from keras.optimizers import RMSprop, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau 
from keras.engine.topology import Layer
from keras import backend as K

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import minmax_scale, StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['X_train.csv', 'sample_submission.csv', 'X_test.csv', 'y_train.csv']


In [2]:
version = 5
basic_name = f'CareerCon_v{version}'
save_model_name_1 = basic_name + '_1.model'
save_model_name_2 = basic_name + '_2.model'
print(basic_name)

CareerCon_v5


In [3]:
train_df = pd.read_csv('../input/X_train.csv')
target_df = pd.read_csv('../input/y_train.csv')
test_df = pd.read_csv("../input/X_test.csv")

In [4]:
# Copied from here
# https://www.kaggle.com/prashantkikani/help-humanity-by-helping-robots

# https://stackoverflow.com/questions/53033620/how-to-convert-euler-angles-to-quaternions-and-get-the-same-euler-angles-back-fr?rq=1
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

def fe_step0(actual):
    
    # https://www.mathworks.com/help/aeroblks/quaternionnorm.html
    # https://www.mathworks.com/help/aeroblks/quaternionmodulus.html
    # https://www.mathworks.com/help/aeroblks/quaternionnormalize.html
        
    actual['norm_quat'] = (actual['orientation_X']**2 + actual['orientation_Y']**2 + actual['orientation_Z']**2 + actual['orientation_W']**2)
    actual['mod_quat'] = (actual['norm_quat'])**0.5
    actual['norm_X'] = actual['orientation_X'] / actual['mod_quat']
    actual['norm_Y'] = actual['orientation_Y'] / actual['mod_quat']
    actual['norm_Z'] = actual['orientation_Z'] / actual['mod_quat']
    actual['norm_W'] = actual['orientation_W'] / actual['mod_quat']
    
    return actual

def fe_step1(actual):
    
    actual['total_angular_velocity'] = (actual['angular_velocity_X'] ** 2 + actual['angular_velocity_Y'] ** 2 + actual['angular_velocity_Z'] ** 2) ** 0.5
    actual['total_linear_acceleration'] = (actual['linear_acceleration_X'] ** 2 + actual['linear_acceleration_Y'] ** 2 + actual['linear_acceleration_Z'] ** 2) ** 0.5
    actual['acc_vs_vel'] = actual['total_linear_acceleration'] / actual['total_angular_velocity']
    
    """Quaternions to Euler Angles"""
    
    x, y, z, w = actual['norm_X'].tolist(), actual['norm_Y'].tolist(), actual['norm_Z'].tolist(), actual['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    
    actual['total_angle'] = (actual['euler_x'] ** 2 + actual['euler_y'] ** 2 + actual['euler_z'] ** 2) ** 5
    actual['angle_vs_acc'] = actual['total_angle'] / actual['total_linear_acceleration']
    actual['angle_vs_vel'] = actual['total_angle'] / actual['total_angular_velocity']
       
    return actual

def fe_step2(actual):
    
    new = actual.copy()
    
    def f1(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    def f2(x):
        return np.mean(np.abs(np.diff(x)))
    
    for col in actual.columns:
        if col in ['row_id', 'series_id', 'measurement_number']:
            continue
        new[col + '_mean'] = actual.groupby(['series_id'])[col].mean()
        new[col + '_min'] = actual.groupby(['series_id'])[col].min()
        new[col + '_max'] = actual.groupby(['series_id'])[col].max()
        new[col + '_std'] = actual.groupby(['series_id'])[col].std()
        new[col + '_max_to_min'] = new[col + '_max'] / new[col + '_min']
        
        # Change. 1st order.
        new[col + '_mean_abs_change'] = actual.groupby('series_id')[col].apply(f2)
        
        # Change of Change. 2nd order.
        new[col + '_mean_change_of_abs_change'] = actual.groupby('series_id')[col].apply(f1)
        
        new[col + '_abs_max'] = actual.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
        new[col + '_abs_min'] = actual.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))

    return new

In [5]:
def delta_feature(df, features):

    for feature in features:
        feat_array = np.append(df[feature].values, df[feature].values[0])
        temp_array = np.zeros(len(feat_array)-1)
        
        for i in range(len(feat_array)-1):
            if i % 128 == 127:
                temp_array[i] =  0
            else:
                temp_array[i] =  feat_array[i+1] - feat_array[i]

        df[feature + '_delta'] = temp_array
    return df

In [6]:
train_df = fe_step0(train_df)
test_df = fe_step0(test_df)

In [7]:
train_df = fe_step1(train_df)
test_df = fe_step1(test_df)

In [8]:
target_features = train_df.columns[3:].values
train_df = delta_feature(train_df, target_features)
test_df = delta_feature(test_df, target_features)

In [9]:
train_df = fe_step2(train_df)
test_df = fe_step2(test_df)

In [10]:
train_df.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,norm_quat,mod_quat,norm_X,norm_Y,norm_Z,norm_W,total_angular_velocity,total_linear_acceleration,acc_vs_vel,euler_x,euler_y,euler_z,total_angle,angle_vs_acc,angle_vs_vel,orientation_X_delta,orientation_Y_delta,orientation_Z_delta,orientation_W_delta,angular_velocity_X_delta,angular_velocity_Y_delta,angular_velocity_Z_delta,linear_acceleration_X_delta,linear_acceleration_Y_delta,linear_acceleration_Z_delta,norm_quat_delta,mod_quat_delta,...,euler_y_delta_mean_abs_change,euler_y_delta_mean_change_of_abs_change,euler_y_delta_abs_max,euler_y_delta_abs_min,euler_z_delta_mean,euler_z_delta_min,euler_z_delta_max,euler_z_delta_std,euler_z_delta_max_to_min,euler_z_delta_mean_abs_change,euler_z_delta_mean_change_of_abs_change,euler_z_delta_abs_max,euler_z_delta_abs_min,total_angle_delta_mean,total_angle_delta_min,total_angle_delta_max,total_angle_delta_std,total_angle_delta_max_to_min,total_angle_delta_mean_abs_change,total_angle_delta_mean_change_of_abs_change,total_angle_delta_abs_max,total_angle_delta_abs_min,angle_vs_acc_delta_mean,angle_vs_acc_delta_min,angle_vs_acc_delta_max,angle_vs_acc_delta_std,angle_vs_acc_delta_max_to_min,angle_vs_acc_delta_mean_abs_change,angle_vs_acc_delta_mean_change_of_abs_change,angle_vs_acc_delta_abs_max,angle_vs_acc_delta_abs_min,angle_vs_vel_delta_mean,angle_vs_vel_delta_min,angle_vs_vel_delta_max,angle_vs_vel_delta_std,angle_vs_vel_delta_max_to_min,angle_vs_vel_delta_mean_abs_change,angle_vs_vel_delta_mean_change_of_abs_change,angle_vs_vel_delta_abs_max,angle_vs_vel_delta_abs_min
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,0.999997,0.999999,-0.758531,-0.634351,-0.10488,-0.10597,0.109076,10.005392,91.728917,2.843273,-0.024668,1.396667,101787.081684,10173.222400,9.331787e+05,0.00000,0.00001,-0.00002,-0.00003,-0.039799,0.012378,0.002618,1.08852,-0.59660,0.3404,-0.000002,-0.000001,...,0.000071,4.008826e-08,0.000203,0.0,-0.000027,-0.000145,0.000085,0.000055,-0.588554,0.000044,3.491367e-08,0.000145,0.0,-5.953182,-131.837332,123.225061,42.237098,-0.934675,29.826396,0.098063,131.837332,0.0,-5.993481,-3827.061862,3450.797163,1038.399784,-0.901683,908.413667,5.476556,3827.061862,0.0,2725.102627,-7.967470e+06,8.003438e+06,1.851575e+06,-1.004514,2.055227e+06,202.507210,8.003438e+06,0.0
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128,0.999995,0.999998,-0.758532,-0.634342,-0.10490,-0.10600,0.074240,9.538638,128.483970,2.843201,-0.024662,1.396651,101763.946161,10668.603678,1.370745e+06,0.00000,-0.00001,-0.00002,0.00003,-0.060576,-0.001005,-0.009364,-0.60424,0.08580,0.6861,0.000011,0.000005,...,0.000112,-4.013931e-08,0.000335,0.0,-0.000028,-0.000362,0.000367,0.000170,-1.013872,0.000116,-3.172025e-07,0.000367,0.0,1.562256,-113.060807,136.726933,38.662807,-1.209322,30.894578,-0.246236,136.726933,0.0,-1.285834,-2512.504396,2582.429117,778.358207,-1.027831,687.382567,-22.870896,2582.429117,0.0,2147.836005,-1.482049e+06,1.256582e+06,3.279269e+05,-0.847868,3.341819e+05,1786.628974,1.482049e+06,0.0
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,1.000006,1.000003,-0.758528,-0.634348,-0.10492,-0.10597,0.030428,8.874697,291.666195,2.843222,-0.024728,1.396677,101773.909064,11467.874519,3.344791e+06,0.00001,-0.00001,-0.00003,0.00000,-0.020328,-0.009486,-0.002995,0.69113,-0.49290,-1.3693,0.000004,0.000002,...,0.000092,-1.153124e-06,0.000272,0.0,-0.000091,-0.000383,0.000055,0.000078,-0.142877,0.000076,4.325032e-07,0.000383,0.0,-39.582251,-621.008373,267.952476,113.520012,-0.431480,77.697661,-1.824020,621.008373,0.0,-84.086305,-9321.668337,6390.147235,2655.466333,-0.685515,2573.799330,11.672745,9321.668337,0.0,1742.417867,-2.242604e+07,2.160548e+07,4.214445e+06,-0.963410

In [11]:
train_df.fillna(0, inplace = True)
test_df.fillna(0, inplace = True)
train_df.replace(-np.inf, 0, inplace = True)
train_df.replace(np.inf, 0, inplace = True)
test_df.replace(-np.inf, 0, inplace = True)
test_df.replace(np.inf, 0, inplace = True)

In [12]:
print(train_df.shape)
print(test_df.shape)

(487680, 503)
(488448, 503)


In [13]:
sc = StandardScaler()  
sc.fit(test_df.drop(["row_id","series_id","measurement_number"], axis=1).values)

onehot_encoder = OneHotEncoder(sparse=False)
y_train_ohe = onehot_encoder.fit_transform((target_df['surface'].values).reshape(-1, 1))
output_shape = y_train_ohe.shape[1]

In [14]:
measurement_df = train_df.loc[train_df['series_id'] == 0]
measurement_df = measurement_df.drop(["row_id","series_id","measurement_number"], axis=1)

measurement_length = max(train_df['measurement_number'].values) + 1
sequence_length = 32
sequences_per_measurement = 7
features = int(measurement_df.shape[1])

training_examples = (max(train_df['series_id'].values) + 1) * sequences_per_measurement
xtrain_dims = [training_examples, sequence_length, features]
xtest_dims = [max(test_df['series_id'].values) + 1, sequence_length, features]

In [15]:
print(xtrain_dims)
print(xtest_dims)

[26670, 32, 500]
[3816, 32, 500]


In [16]:
X_train = np.zeros(xtrain_dims, dtype=float)
y_train = np.empty([training_examples, y_train_ohe.shape[1]])
X_temp = []

for i in range(max(train_df['series_id'].values) + 1):
    measurement_df = train_df.loc[train_df['series_id'] == i]
    measurement_df = measurement_df.drop(["row_id","series_id","measurement_number"], axis=1)
    X_temp = sc.transform(measurement_df.values)
    j=i*sequences_per_measurement
    X_train[j] = X_temp[0:32]
    X_train[j+1] = X_temp[32:64]
    X_train[j+2] = X_temp[64:96]
    X_train[j+3] = X_temp[96:128]
    X_train[j+4] = X_temp[16:48]
    X_train[j+5] = X_temp[48:80]
    X_train[j+6] = X_temp[80:112]
    y_train[j] = y_train_ohe[i]
    y_train[j+1] = y_train_ohe[i]
    y_train[j+2] = y_train_ohe[i]
    y_train[j+3] = y_train_ohe[i]
    y_train[j+4] = y_train_ohe[i]
    y_train[j+5] = y_train_ohe[i]
    y_train[j+6] = y_train_ohe[i]


In [17]:
print(X_train.shape)
print(y_train.shape)

(26670, 32, 500)
(26670, 9)


In [18]:
X_test0 = np.zeros(xtest_dims, dtype=float)
X_test1 = np.zeros(xtest_dims, dtype=float)
X_test2 = np.zeros(xtest_dims, dtype=float)
X_test3 = np.zeros(xtest_dims, dtype=float)

for i in range(xtest_dims[0]):
    measurement_df = test_df.loc[test_df['series_id'] == i]
    measurement_df = measurement_df.drop(["row_id","series_id","measurement_number"], axis=1)
    X_temp = sc.transform(measurement_df.values)
    X_test0[i] = X_temp[0:32]
    X_test1[i] = X_temp[32:64]
    X_test2[i] = X_temp[64:96]
    X_test3[i] = X_temp[96:128]

del measurement_df, train_df, test_df

In [19]:
print(X_test0.shape)

(3816, 32, 500)


In [20]:
def combined_global_pool(blockInput):
    max_pool = GlobalMaxPooling1D()(blockInput)
    avg_pool = GlobalAveragePooling1D()(blockInput)
    return concatenate([avg_pool, max_pool])

def combined_pool(blockInput):
    max_pool = MaxPooling1D(pool_size=3, strides=None)(blockInput)
    avg_pool = AveragePooling1D(pool_size=3, strides=None)(blockInput)
    return concatenate([avg_pool, max_pool])

def build_model_lstm(sequence_length, features, output_size, lstm_units, dense_units, dropout):
    
    imp_layer = Input(shape=(sequence_length, features))
    
    x = Bidirectional(CuDNNLSTM(lstm_units, return_sequences=True, kernel_initializer=glorot_normal(seed=1999), recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(imp_layer)
    x = Bidirectional(CuDNNLSTM(lstm_units, return_sequences=True, kernel_initializer=glorot_normal(seed=12), recurrent_initializer=orthogonal(gain=1.0, seed=13)))(x)
    x = combined_global_pool(x)
    x = BatchNormalization()(x) 
    x = Dropout(2*dropout)(x) 
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout)(x)   
    out_layer = Dense(output_size, activation="softmax")(x)
    
    model = Model(inputs=imp_layer, outputs=out_layer)
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy']) 
    
    return model

def build_model_gru(sequence_length, features, output_size, gru_units, conv_units, dense_units, dropout):
    
    imp_layer = Input(shape=(sequence_length, features))
    
    x = Bidirectional(CuDNNGRU(gru_units, return_sequences = True))(imp_layer)
    x_conv1 = Conv1D(conv_units, kernel_size = 2, padding = "same", kernel_initializer = "he_uniform")(x)
    x_conv2 = Conv1D(conv_units, kernel_size = 3, padding = "same", kernel_initializer = "he_uniform")(x)
    x = concatenate([x_conv1, x_conv2])
    x = combined_global_pool(x)
    x = BatchNormalization()(x) 
    x = Dropout(2*dropout)(x) 
    x = Dense(dense_units, activation="relu")(x)
    x = Dropout(dropout)(x)
    out_layer = Dense(output_size, activation="softmax")(x)
    
    model = Model(inputs=imp_layer, outputs=out_layer)
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [21]:
model1 = build_model_lstm(sequence_length=sequence_length, features=features, output_size=output_shape, lstm_units=128, dense_units=64, dropout=0.2)
#model2 = build_model_gru(sequence_length=sequence_length, features=features, output_size=output_shape, gru_units=128, conv_units=64, dense_units=32, dropout=0.2)
#model2.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
reduce_lr1 = ReduceLROnPlateau(monitor='val_acc', mode = 'max', factor=0.5, patience=3, min_lr=0.00005, verbose=1)
#reduce_lr2 = ReduceLROnPlateau(monitor='val_acc', mode = 'max', factor=0.5, patience=3, min_lr=0.00005, verbose=1)
model_checkpoint1 = ModelCheckpoint(save_model_name_1, save_best_only=True, verbose=1)
#model_checkpoint2 = ModelCheckpoint(save_model_name_2, save_best_only=True, verbose=1)

In [23]:
epochs = 10
num_folds = 3
loops = 2 
    
for j in range(loops):
    kfold = KFold(n_splits = num_folds, shuffle=False, random_state=None)

    for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
        print("Loop:", j,"Model: 1, Fold:",i) 
        history1 = model1.fit(X_train[train_index], y_train[train_index], batch_size=32, validation_data=[X_train[test_index], y_train[test_index]], callbacks=[model_checkpoint1, reduce_lr1], epochs=epochs, verbose=10) 
    
        #print("Loop:", j,"Model: 2, Fold:",i) 
        #history2 = model2.fit(X_train[train_index], y_train[train_index], batch_size=32, validation_data=[X_train[test_index], y_train[test_index]], callbacks=[model_checkpoint2, reduce_lr2], epochs=epochs, verbose=10) 



Loop: 0 Model: 1, Fold: 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 17780 samples, validate on 8890 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 1.56112, saving model to CareerCon_v5_1.model
Epoch 2/10

Epoch 00002: val_loss improved from 1.56112 to 1.46925, saving model to CareerCon_v5_1.model
Epoch 3/10

Epoch 00003: val_loss improved from 1.46925 to 1.11437, saving model to CareerCon_v5_1.model
Epoch 4/10

Epoch 00004: val_loss improved from 1.11437 to 1.05406, saving model to CareerCon_v5_1.model
Epoch 5/10

Epoch 00005: val_loss did not improve from 1.05406
Epoch 6/10

Epoch 00006: val_loss did not improve from 1.05406
Epoch 7/10

Epoch 00007: val_loss did not improve from 1.05406
Epoch 8/10

Epoch 00008: val_loss did not improve from 1.05406
Epoch 9/10

Epoch 00009: val_loss did not improve from 1.05406
Epoch 10/10

Epoch 00010: val_loss did not improve from 1.05406
Loop

In [24]:
y_pred_m1 = model1.predict(X_train)
#y_pred_m2 = model2.predict(X_train)
y_true = (np.argmax(y_train, axis=1)).reshape(-1, 1)

In [25]:
confusion_matrix(y_true, (np.argmax(y_pred_m1, axis=1)).reshape(-1, 1))

array([[1294,    0,    0,    0,    0,    0,    1,   28,    0],
       [   0, 5392,   14,    0,    0,    4,    2,   33,    8],
       [   0,    0, 2527,    0,    0,    0,    0,   14,    0],
       [   0,    0,    0,  140,    0,    0,    0,    7,    0],
       [   0,    0,    0,    0, 2137,    0,    0,   19,    0],
       [   0,    0,    0,    0,    0, 5104,    6,    9,    5],
       [   0,    0,    0,    0,    0,    0, 2037,   42,    0],
       [   0,    0,    0,    0,    0,    0,    0, 3592,    6],
       [   0,    3,    0,    0,    0,    0,    3,    0, 4243]])

In [26]:
#confusion_matrix(y_true, (np.argmax(y_pred_m2, axis=1)).reshape(-1, 1))

In [27]:
#confusion_matrix(y_true, (np.argmax((y_pred_m1 + y_pred_m2), axis=1)).reshape(-1, 1))

In [28]:
y_test = model1.predict(X_test0)
y_test += model1.predict(X_test1)
y_test += model1.predict(X_test2)
y_test += model1.predict(X_test3)
#y_test += model2.predict(X_test0)
#y_test += model2.predict(X_test1)
#y_test += model2.predict(X_test2)
#y_test += model2.predict(X_test3)
y_test /= 4

In [29]:
submission = pd.read_csv("../input/sample_submission.csv")
submission['surface'] = onehot_encoder.inverse_transform(y_test)
submission.to_csv('submision.csv', index=False)